# Function calling

Function calling is a powerful feature of OpenAI `gpt-4` and other models. It essentially is a **Function Picker and Parameter Provider**. It can help you choose which function, if any, to invoke with which parameters, under provided context and instruction. 

LionAGI allows simple usage of function callings in the `Session` object. 

In [1]:
import lionagi as li

In [2]:
# let us define a function description in OpenAI schema

tools = [
    {
        "type": "function",
        "function": {
            "name": "multiply",
            "description": "Perform multiplication on two numbers",
            "parameters": {
                "type": "object",
                "properties": {
                    "number1": {
                        "type": "number",
                        "description": "a number to multiply, e.g. 5.34",
                    },
                    "number2": {
                        "type": "number",
                        "description": "a number to multiply, e.g. 17",
                    },
                },
                # specify which parameters are required for the model to respond when function calling
                "required": ["number1", "number2"],
            },
        }
    }
]

def multiply(number1, number2):
    return number1*number2

In [3]:
# and now some messages
system = "you are asked to perform as a function picker and parameter provider"
task = "Think step by step, understand the following basic math question and provide two numbers as parameters for function calling."

# when using respond_mode as json to enforce output format, you need to provide specifying details in instruction
json_format = {"number1": "x", "number2": "y"}

instruct1 = {"Task": task, "json_format": json_format}

In [4]:
question = "There are [basketball, football, backpack, water bottle, strawberry, tennis ball, rockets]. each comes in four different colors, what is the number of unique kinds of ball?"

context1 = {"Question": question}

In [5]:
session = li.Session(system=system)

session.register_tools(tools, multiply)
session.llmconfig.update({
    "tools": tools, 
    "temperature":0.35,
    "tool_choice": "auto", 
    "response_format": {'type':'json_object'}
})

In [6]:
response = await session.initiate(instruction=instruct1, context=context1)
print(response)


{
  "tool_uses": [
    {
      "recipient_name": "functions.multiply",
      "parameters": {
        "number1": 3,  "number2": 4
      }
    }
  ]
}


In [7]:
session.conversation.messages[-1]

{'role': 'tool', 'content': 12}